### 모델 생성 및 데이터 전처리

In [2]:
import numpy as np
import os
import cv2
from tensorflow.keras import applications, models, layers

In [171]:
# 데이터 전처리
def get_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    for i in range(len(dir_list)-1):
        dir_path = path + "/" + dir_list[i+1]
        dir_name = os.listdir(dir_path)

        for j in range(len(dir_name)):
            full_dir_path = dir_path + "/" + dir_name[j]

            img = cv2.imread(full_dir_path)
            img = cv2.resize(img, (32,32))
            x.append(img)
            y.append(i)
    # print(x)
    # print(y)
    return np.array(x),np.array(y)
#get_data()

In [200]:
def process_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    #print(os.listdir(path))
    for i in range(0,len(dir_list)):
        # if i == 1:
        #     break
        dir_path = path + "/" + dir_list[i]
        #print(os.listdir(dir_path))
        dir_name = os.listdir(dir_path)
        
        for j in range(len(dir_name)):
            
            full_dir_path = dir_path + "/" + dir_name[j]
            print(full_dir_path)
            img = cv2.imread(full_dir_path,cv2.IMREAD_COLOR)
            
            # 색 변경
            img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
            lower = np.array([0,48,80], dtype="uint8")
            upper = np.array([20,255,255], dtype="uint8")
            img_hand = cv2.inRange(img_hsv,lower,upper)
            

            #경계선 찾음
            contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            # 가장 큰 영역 찾기
            max = 0
            maxcnt = None
            
            
            
            for cnt in contours :
                area = cv2.contourArea(cnt)
                if(max < area) :
                    max = area
                    maxcnt = cnt
            #print("maxcnt : ",maxcnt)
            if maxcnt != np.array([]):
                mask = np.zeros(img.shape).astype(img.dtype)
                #print(mask)
                # 경계선 내부 255로 채우기
                color = [255, 255, 255]
                cv2.fillPoly(mask, [maxcnt], color)
                img_hand = cv2.bitwise_and(img, mask)
                
                
                img_hand = cv2.resize(img_hand, (100,100))
                cv2.imwrite('image.png', img_hand)
                x.append(img_hand)
                y.append(i)
            else:
                continue
            
    #print(x.shape)
    #print(y.shape)
    return np.array(x),np.array(y)

#process_data("./data/train")


In [208]:
# 모델 생성
def make_model() :
    resnet50 = applications.resnet50.ResNet50(include_top=False, weights="imagenet", input_shape=(100,100,3))
    resnet50.trainable = False
    model = models.Sequential()
    model.add(resnet50)
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(512, activation="relu"))
    #model.add(layers.Dense(256, activation="relu"))

    # 분류 데이터가 몇종류인지
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
    print(model.summary())
    return model

In [174]:
# 한개 테스트용
def get_test():
    test_path = "./test/non/test17.png"
    test_img = cv2.imread(test_path)
    #print(test_img)
    test_img = cv2.resize(test_img, (32,32))
    #test_img = test_img.reshape(32,32,3)
    test_list = []
    test_list.append(test_img)
    test_list = np.array(test_list)
    #print(test_img.shape)
    cv2.imwrite("test123.png",test_img)
    return test_list

In [213]:
# 모델 학습
def learning() :
    train_path = "./data_image"
    test_path = "./test"
    (x_train, y_train)= process_data(train_path)
    (x_test, y_test) = process_data(test_path)
    model = make_model()
    # print("x_train : ",x_train[0])
    # print("x_train.shape : ",x_train[0].shape)
    log = model.fit(x_train, y_train, epochs=2, batch_size=16)
    #model.save("face.h5")
    
    model.evaluate(x_test,y_test)
learning()

./data/train/non/176.png


/var/folders/xl/l17__8q55nsdyt859lgd1nvc0000gn/T/ipykernel_50052/95893456.py:43: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if maxcnt != np.array([]):


./data/train/non/162.png
./data/train/non/189.png
./data/train/non/77.png
./data/train/non/63.png
./data/train/non/62.png
./data/train/non/188.png
./data/train/non/76.png
./data/train/non/163.png
./data/train/non/177.png
./data/train/non/89.png


/var/folders/xl/l17__8q55nsdyt859lgd1nvc0000gn/T/ipykernel_50052/95893456.py:43: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if maxcnt != np.array([]):


./data/train/non/149.png
./data/train/non/161.png
./data/train/non/175.png
./data/train/non/60.png
./data/train/non/74.png
./data/train/non/48.png
./data/train/non/49.png
./data/train/non/75.png
./data/train/non/61.png
./data/train/non/174.png
./data/train/non/160.png
./data/train/non/148.png
./data/train/non/164.png
./data/train/non/170.png
./data/train/non/158.png
./data/train/non/59.png
./data/train/non/65.png
./data/train/non/71.png
./data/train/non/70.png
./data/train/non/64.png
./data/train/non/58.png
./data/train/non/159.png
./data/train/non/171.png
./data/train/non/165.png
./data/train/non/173.png
./data/train/non/167.png
./data/train/non/99.png
./data/train/non/8.png
./data/train/non/72.png
./data/train/non/66.png
./data/train/non/67.png
./data/train/non/73.png
./data/train/non/9.png
./data/train/non/166.png
./data/train/non/98.png
./data/train/non/172.png
./data/train/non/129.png
./data/train/non/115.png
./data/train/non/101.png
./data/train/non/14.png
./data/train/non/28.png

2022-06-30 21:39:38.660665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 7s 127ms/step - loss: 4.4286 - accuracy: 0.8966
Epoch 2/10
24/24 [==============================] - 3s 104ms/step - loss: 3.4087 - accuracy: 0.9576
Epoch 3/10
24/24 [==============================] - 2s 97ms/step - loss: 0.5454 - accuracy: 0.9708
Epoch 4/10
24/24 [==============================] - 2s 95ms/step - loss: 0.5587 - accuracy: 0.9841
Epoch 5/10
24/24 [==============================] - 2s 93ms/step - loss: 0.0917 - accuracy: 0.9894
Epoch 6/10
24/24 [==============================] - 2s 97ms/step - loss: 0.0906 - accuracy: 0.9920
Epoch 7/10
24/24 [==============================] - 2s 91ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 8/10
24/24 [==============================] - 2s 93ms/step - loss: 3.3815e-04 - accuracy: 1.0000
Epoch 9/10
24/24 [==============================] - 2s 95ms/step - loss: 2.4609e-06 - accuracy: 1.0000
Epoch 10/10
24/24 [==============================] - 2s 93ms/step - loss: 7.6032e-07 - accuracy: 1.0000


2022-06-30 21:40:03.912301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 2s 390ms/step - loss: 83.9067 - accuracy: 0.5591


In [193]:
import cv2
import numpy as np
path = "./tracking/test1.png"

img = cv2.imread(path, cv2.IMREAD_COLOR)

# 색 변경
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

img_hand = cv2.inRange(img_gray,(10), (165))
mask = np.zeros(img.shape).astype(img.dtype)

#경계선 찾음
contours, hierarchy = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 영역 찾기
max = 0
maxcnt = None
for cnt in contours :
    area = cv2.contourArea(cnt)
    if(max < area) :
        max = area
        maxcnt = cnt


# 경계선 내부 255로 채우기
color = [255, 255, 255]
cv2.fillPoly(mask, [maxcnt], color)
img_hand = cv2.bitwise_and(img, mask)
hull = cv2.convexHull(maxcnt)
#print(hull)

# 도출된 값으로 사각형 그리기
x,y,w,h = cv2.boundingRect(hull)
cv2.rectangle(mask, (x,y), (x+w,y+h), (0,255,0),3 )


cv2.imshow('image', mask)
#cv2.imshow('image', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

### 드론

In [1]:
import cv2
import numpy as np
from djitellopy import Tello

from tensorflow.keras import models

In [2]:
# 배터리 체크
def battery_check() : 
  drone = Tello()
  drone.connect()

  power = drone.get_battery()
  if power < 30 : print("배터리 부족", power)q
  else : print("배터리", power)
  drone.end()
  return power
battery_check()

NameError: name 'Tello' is not defined

In [2]:
#  -- 초기값 설정 --
# 스타트flag
start_counter = 0
# 프레임 flag - 이때 딥러닝 예측도 실행해야됨
frame_flag = 70
# 임계값 조절
tolerance_x = 5
tolerance_y = 5
# 속도 한계값
slowdown_threshold_x = 15
slowdown_threshold_y = 15
# 속도
drone_speen_x = 10
drone_speen_y = 10
# 화면 조절
set_point_x = 960/2
set_point_y = 720/2
hull = 0
# x,y,w,h 전역변수
# global x, y, w, h
x =0
y =0
w =0
h =0
accuracy_num = ''

# 드론 초기화
drone = Tello()
drone.connect()

# # 스트림 연결 오류 
drone.streamon()

# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,1200)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1600)
# 모델 임포트
# faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = models.load_model("last_model.h5")
#x,y,w,h = 0
# ------------ 영상 루프 시작 ------------
while True :
  # takeoff 설정
  if start_counter == 0 :
    drone.takeoff()
    print('takeoff')
    start_counter = 1

  # 프레임 읽어오기
  frame = drone.get_frame_read().frame
  # _, frame = cap.read()

  cv2.circle(frame, (int(set_point_x), int(set_point_y)), 12, (255,0,0), 1) # 화면 중간에 원표시
  cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3 )
  # cv2.putText(frame,f'accuracy = ',accuracy_num,(x+5,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 0.5, cv2.LINE_AA)
  # flalg 설정
  frame_flag -=1
  if frame_flag < 0:
    frame_flag =70
  #print(frame_flag)
  # print("frame_flag", frame_flag)
  if frame_flag == 0 :
    # ------------ 데이터 가공 ------------
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    img_hsv = cv2.fastNlMeansDenoisingColored(img_hsv,None,10,10,7,21)
    lower = np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
    img_hand = cv2.inRange(img_hsv,lower,upper)
    
    #경계선 찾음
    contours, _ = cv2.findContours(img_hand, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # 가장 큰 영역 찾기
    max = 0
    maxcnt = None
    
    # 채워주기
    for cnt in contours :
      area = cv2.contourArea(cnt)
      if(max < area) :
        max = area
        maxcnt = cnt
    
    # 예외처리
    if maxcnt != np.array([]):
      mask = np.zeros(frame.shape).astype(frame.dtype)
      # 경계선 내부 255로 채우기
      color = [255, 255, 255]
      cv2.fillPoly(mask, [maxcnt], color)
      hull = cv2.convexHull(maxcnt)
      x, y, w, h = cv2.boundingRect(hull)
      print("x,y,w,h : ", x,y,w,h)
      cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3 )
      cv2.circle(frame, (int(x+w/2) , int(y+h/2)), 12, (0,255,0), 1) # 얼굴 중심 표시
      img_hand = cv2.bitwise_and(frame, mask)

      #print("전",img_hand.shape)
      img_hand = cv2.resize(img_hand, (100,100))
      #print(img_hand.shape)
    else :
      continue
    # ------------ 예측 실행 ------------
    img_hand = np.array([img_hand])
    pred = model.predict(img_hand)
    print("예측 값 ==>  ",pred)
    # accuracy_num = str(pred[0][1])
    print(accuracy_num)
    pred = np.argmax(pred)
    print("1 : 얼굴 인식,  0 : 인식 실패 ==> ",pred)
    
    if pred == 1 :
      
      frame_flag = 100

      # 도출된 값으로 사각형 그리기
      

      # ------------ 드론 제어 ------------
      #  얼굴중심과 화면중심의 차를 계산
      distance_x = x+w/2 - set_point_x
      distance_y = y+h/2 - set_point_y

      up_down_velocity = 0
      right_left_veiocity = 0
      for_back_veiocity = 0

    # 드론 좌우 이동
      if distance_x < -tolerance_x:
        print("left move")
        right_left_veiocity = - drone_speen_x
      elif distance_x > tolerance_x:
        print("right move")
        right_left_veiocity = drone_speen_x
      else :
        print("OK")

      # 드론 상하 이동
      if distance_y < -tolerance_y:
        print("up move")
        up_down_velocity = drone_speen_y
      elif distance_y > tolerance_y:
        print("down move")
        up_down_velocity = - drone_speen_y
      else :
        print("OK")

      # 드론 앞뒤 이동 및 프레임 크면 넘기기
      if w*h < 960*720/2:
        for_back_veiocity = 10
      elif w*h > 960*720/2:
        for_back_veiocity = -10
      elif w*h > 960*720 :
        for_back_veiocity = 10
      elif w*h >= 950*700 :
        continue
      else:
        print("OK")

      #  임계치 이상 벗어나면 속도 조정 
      if abs(distance_x) < slowdown_threshold_x:
        right_left_veiocity = int(right_left_veiocity / 2)
      if abs(distance_y) < slowdown_threshold_y:
        up_down_velocity = int(up_down_velocity / 2)

      #드론 움직이기
      drone.send_rc_control(right_left_veiocity, 0, up_down_velocity, 0)
    else :
      # drone.move_left(20)
      print("0일때 물체 추적")
      continue

  # 비디오 띄우기
  cv2.imshow("Video", frame)
  # 키 설정
  key = cv2.waitKey(1)
  if key == ord('q'):
    break


drone.streamoff()
cv2.destroyAllWindows()
drone.end()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'
[INFO] tello.py - 437 - Send command: 'takeoff'
[WARNING] tello.py - 447 - Aborting command 'takeoff'. Did not receive a response after 20 seconds
[INFO] tello.py - 437 - Send command: 'takeoff'
